## Using TensorFlow to deal with iris data

### 1 Create a simple graph and run it:  $f(x,y) = x^2\cdot y + y + 2$

In [25]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [26]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
sess.run(f)

42

In [27]:
sess.close()

In [28]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result

42

### 2 Classify iris data using Logistic Regression by TensorFlow

#### 2.1 Normal Equation: $\theta = (X^T\cdot X)^{-1}\cdot X^T\cdot y$

Note that housing.target is a 1D array, but we need to reshape it to a column vector to compute theta.

In [29]:
import numpy as np
from sklearn.datasets import load_iris

dataset = load_iris()
feature = dataset.data
m, n = feature.data.shape
label = dataset.target

In [30]:
X = tf.constant(feature, dtype=tf.float32, name="X")
y = tf.constant(label.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
error = tf.matmul(X, theta) - y
mse = tf.reduce_mean(tf.square(error), name="mse")

with tf.Session() as sess:
    theta_result = theta.eval()
    mse_result = mse.eval()
    
mse_result

0.046666786

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_feature = scaler.fit_transform(feature)

#### 2.2 Using Batch Gradient Descent to fit iris data
Gradient: $ \nabla_{\theta}MSE(\theta) = \frac{2}{m}X^T\cdot(X\cdot\theta - y) $ <br>
Gradient Descent step: $ \theta^{(next step)} = \theta - \eta\cdot \nabla_{\theta}MSE(\theta) $

In [24]:
np.random.seed(42)

X = tf.constant(feature, dtype=tf.float32, name="X")
y = tf.constant(label.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n, 1], -1.0, 1.0, seed=42), name="theta")
error = tf.matmul(X, theta) - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), tf.matmul(X, theta) - y)
training = tf.assign(theta, theta - 0.01 * gradients)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        sess.run(training)
    best_mse = mse.eval()
    
best_mse

0.046908583